### Improving Computer Vision Accuracy using Convolutions

##### 1. Image Detection without using Convolutions:
Let's start by making a simple Neural Network using Tensorflow and keras. There is a whole Jupyter notebook with more details about this, so feel free to check it out!
Later we are going to use the accuracy, loss and run-time of our simple model to compare with our Neural Network with convolutions implemented! So stay tuned..

In [1]:
import tensorflow as tf

#Accessing data
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

#Scaling the input sets
training_images=training_images / 255.0
test_images=test_images / 255.0

#Creating the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)

#Evaluating the model
test_loss = model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5000 - accuracy: 0.8253
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3750 - accuracy: 0.8662
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3374 - accuracy: 0.8774
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3125 - accuracy: 0.8861
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3460 - accuracy: 0.8758


So our training model accuracy is 89%, and we got around 87% for our testing image set. Although these are some good results let's see how using convolutions can improve each aspect!(or maybe not!)

#### 2. Image Detection using Convolutions

I'm going to start from scratch!
I will import tensorflow, then will get my data from fashion_mnist.
Then I have to reshape my data!If you don't do this, you'll get an error when training as the Convolutions do not recognize the shape."That's because the first convolution expects a single tensor containing everything, so instead of 60,000 28x28x1 items in a list, we have a single 4D list that is 60,000x28x28x1, and the same for the test images." 

In [2]:
import tensorflow as tf
print("TensotFlow Version:", tf.__version__)

#Getting the data
mnist=tf.keras.datasets.fashion_mnist
(training_image, training_label), (testing_image, testing_label)=mnist.load_data()

#Reshaping input data
training_image=training_image/255
testing_image=testing_image/255

training_image= training_image.reshape(60000, 28, 28, 1)
testing_image= testing_image.reshape(10000, 28, 28, 1)

TensotFlow Version: 2.2.0


Now we're going to create the model. Instead of starting with the input layer, here we're going to start with a convolution! Then we'll follow each by a pooling layer (to compress the image for the next layer!). For example, the convolution looks something like:

*tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1))*

* So 32 is the arbitrary number of your filters! 32 is a good number to start.
* (3, 3) is the size of the convolution. Here is a 3 X 3 grid.
* The activation function here is 'relu' which basically returns a positive output.
* Last but not least is the shape of the input image; which here is 28 X 28 pixels.

After each convolution comes a pooling:

*tf.keras.layers.MaxPooling2D((2,2))*

* By choosing (2,2) we are reducing each image by 1/4 (25%).

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'), #adding another convolution followed by pooling
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(), #flattening the output. from here is the same simple NN
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [4]:
#Compiling out model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
#Calling the fit method to do the training
model.fit(training_image, training_label, epochs=5)

#Evaluating the model
test_loss, test_acc = model.evaluate(testing_image, testing_label)
print(test_acc)

Epoch 1/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.4365 - accuracy: 0.84130s - l
Epoch 2/5
1875/1875 [==============================] - 58s 31ms/step - loss: 0.2920 - accuracy: 0.8932
Epoch 3/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.2475 - accuracy: 0.90840s - loss: 0.247
Epoch 4/5
1875/1875 [==============================] - 58s 31ms/step - loss: 0.2150 - accuracy: 0.9205
Epoch 5/5
313/313 [==============================] - 3s 9ms/step - loss: 0.2618 - accuracy: 0.9060
0.906000018119812


Although the training took longer, the accuracy of the training model increased to 92% and 91% when using our testing set!
We can improve the accuracy by changing the number of epoch however there are always chances of overfitting with higher epochs which we need to keep in mind!

#### 3. Implementing a "callback"

Here I am basically going to repeat the previous NN using convolution using the mnist dataset. I will only use 32 filters, and will set the epoch to 10! However I will use a callback function so that the training stops when I get to 99% accuracy! 

In [6]:
#Getting our data together!
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0

test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

#Callback function
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')<=0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training=True
            
callbacks=MyCallback()


#Making the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

Epoch 1/10
1875/1875 [==============================] - ETA: 0s - loss: 0.1440 - accuracy: 0.9567 - ETA: 0s - loss: 0.1444 - accuracy
Reached 99% accuracy so cancelling training!
313/313 [==============================] - 2s 6ms/step - loss: 0.0733 - accuracy: 0.9765
0.9764999747276306


At the end of the epoch when the accuracy reached 99% the training was stopped! So we didn't have to wait till all 10 epochs are over!

Hope you enjoyed this project! I really recommend taking the Coursera "Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning". Laurance Moroney does a great job explaining ins and outs of each neural network!

Cheers!